In [ ]:
Code crafted by: Yamid Felipe Quiroga González

In [1]:
#IMPORTED LIBRARIES
#requests is a library that allows us to make requests to APIS, this is essential for the project
#json is a library to modify the visual appearance of a json
#datetime is a library that contains time values, its use is mainly related to data access
#azurecosmos are libraries for connecting to the cosmos database in Azure, these allow us to access the database API to make queries
#os is a library that allows us to interact with the operating system where the program is executed
import json, requests, os
from datetime import datetime
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime
from windows_tools.installed_software import get_installed_software

In [41]:
# settings: Dictionary containing configurations for connecting to the Cosmos database on Azure.
#   - host: Address of the database host. Defaults to 'https://yamiddb.documents.azure.com:443/' if not provided in the 'ACCOUNT_HOST' environment variable.
#   - master_key: Master access key to the database. Defaults to the provided key if none is specified in the 'ACCOUNT_KEY' environment variable.
#   - database_id: ID of the database to connect to. Defaults to 'Vulnerabilities' if not provided in the 'COSMOS_DATABASE' environment variable.
#   - container_id: ID of the container in the database. Defaults to 'SoftwareList' if not provided in the 'COSMOS_CONTAINER' environment variable.

#Delete the """ to run the code
"""
settings = {
    'host': os.environ.get('ACCOUNT_HOST', ''),
    'master_key': os.environ.get('ACCOUNT_KEY', ''),
    'database_id': os.environ.get('COSMOS_DATABASE', 'Vulnerabilities'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'SoftwareList'),
}

# HOST: Address of the Cosmos database host on Azure.
HOST = settings['host']

# MASTER_KEY: Master access key to the Cosmos database on Azure.
MASTER_KEY = settings['master_key']

# DATABASE_ID: ID of the database to connect to.
DATABASE_ID = settings['database_id']

# CONTAINER_ID: ID of the container in the database.
CONTAINER_ID = settings['container_id']
"""

In [3]:
# Method to create or validate the information of cosmos database
def run_sample():
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


if __name__ == '__main__':
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")

Database with id 'Vulnerabilities' was found
Container with id 'SoftwareList' was found

run_sample done


In [35]:
"""
Retrieves vulnerability information from the CVE Details API based on the specified software product ID and version ID.

Args:
    sw (str): The product ID of the software.
    vs (str): The version ID of the software.

Returns:
    dict: A dictionary containing vulnerability information retrieved from the API in JSON format.
"""
def apiVulnDb(sw, vs):

    url = f"https://www.cvedetails.com/api/v1/vulnerability/search?outputFormat=json&productId={sw}&versionId={vs}&pageNumber=1&resultsPerPage=20"

    payload = {}
    headers = {
        'accept': '',
        'accept-language': '',
        'cache-control': '',
        'cookie': '',
        'if-modified-since': '',
        'referer': '',
        'sec-ch-ua': '',
        'sec-ch-ua-mobile': '',
        'sec-ch-ua-platform': '',
        'sec-fetch-dest': '',
        'sec-fetch-mode': '',
        'sec-fetch-site': '',
        'sec-fetch-user': '',
        'upgrade-insecure-requests': '',
        'user-agent': '',
        'Authorization': ''
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()


In [42]:
"""
Retrieves information about a specific CVE (Common Vulnerabilities and Exposures) entry from the MITRE CVE database.

Args:
    cve (str): The CVE ID to retrieve information for.

Returns:
    dict: A dictionary containing information about the specified CVE entry in JSON format.

"""
def apiCVE(cve):
    return requests.get(f"https://cveawg.mitre.org/api/cve/{str(cve)}").json()

In [6]:
"""
Retrieves the list of affected products for a given CVE (Common Vulnerabilities and Exposures) entry.

Args:
    cve (str): The CVE ID to retrieve affected products for.

Returns:
    list or str: A list containing the names of affected products, or 'not found' if no affected products are found.

"""
def get_affected_products(cve):

    # Load the JSON data for the CVE
    dataCVE = apiCVE(cve)

    # Get the list of affected products from the JSON data
    affected_products = dataCVE['containers']['cna']['affected']

    # Create a list to store the product names
    product_list = []

    # Iterate over the list of affected products and extract the product names
    for affected_product in affected_products:
        product_list.append(affected_product['product'])

    # Return the list of products or 'not found' if the list is empty
    return product_list if product_list else 'not found'

In [7]:
"""
Retrieves CVSS (Common Vulnerability Scoring System) metrics for a given CVE (Common Vulnerabilities and Exposures) entry from the OpenCVE API.

Args:
    cve (str): The CVE ID to retrieve CVSS metrics for.

Returns:
    dict: A dictionary containing CVSS metrics for the specified CVE entry in JSON format.
"""
def apiCVEmetric(cve):

    payload = {}
    headers = {
        'Authorization': ''
    }
    url = f"https://www.opencve.io/api/cve/{cve}"
    response = requests.request("GET", url, headers=headers, data=payload)
    return response.json()


In [43]:
"""
Retrieves combined CVSS metrics for a given CVE (Common Vulnerabilities and Exposures) entry from the OpenCVE API.

Args:
    cve (str): The CVE ID to retrieve combined CVSS metrics for.

Returns:
    dict: A dictionary containing combined CVSS metrics for the specified CVE entry.

"""
def getMetric(cve):

    response = apiCVEmetric(cve)

    # Create a new dictionary to store the combined metrics
    new_metrics = {}

    # Check if the key "metrics" exists in the original JSON
    if "metrics" in response["raw_nvd_data"]:
        # Check if the key "cvssMetricV2" exists in "metrics"
        if "cvssMetricV2" in response["raw_nvd_data"]["metrics"]:
            new_metrics = response["raw_nvd_data"]["metrics"]["cvssMetricV2"]
        else:
            cvssMetricV2_data = []
        
        # Check if the key "cvssMetricV31" exists in "metrics"
        if "cvssMetricV31" in response["raw_nvd_data"]["metrics"]:
            new_metrics = response["raw_nvd_data"]["metrics"]["cvssMetricV31"]
        else:
            cvssMetricV31_data = []
    else:
        # If the key "metrics" does not exist, initialize both lists as empty
        cvssMetricV2_data = []
        cvssMetricV31_data = []
    
    return new_metrics

In [44]:
"""
Saves a JSON document in the Cosmos DB container.

Args:
    Swjson (dict): The JSON document to be saved.
"""
def saveJSON(Swjson):
    # Save the json in the container
    try:
        container = db.get_container_client(CONTAINER_ID)
        container.create_item(body = Swjson)
        print("Items already saved")
    except exceptions.CosmosResourceExistsError:
        print("Something wrong saving items")

In [36]:
"""
Searches for vulnerabilities associated with a specific software product and version,
retrieves relevant information, and saves it in the Cosmos DB container.

Args:
    product (str): The name of the software product.
    version (str): The version of the software product.

"""
def seachSWVulnerabilities(product, version):

    response = apiVulnDb(product, version)
    
    # Build the JSON document to be saved in the DB
    swVulnerabilities = {
        'id': f"{response['results'][0]['product']}_{version}_{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        'partitionKey': 'partitionKey',
        "vulnerabilities": {} 
    }
    
    # Search in the VulnDB API response for CVEs and their titles to add this information to the new JSON
    for result in response['results']:
        cve_id = result['cveId']
        print(cve_id)
        swVulnerabilities["vulnerabilities"][cve_id] = {
            "summary": result["summary"],
            "affect": get_affected_products(cve_id), 
            "metrics": getMetric(cve_id)
        }

    # Save data
    saveJSON(swVulnerabilities)
    
    # Print the JSON result
    print(json.dumps(swVulnerabilities, indent=4))

In [40]:
seachSWVulnerabilities("320","684235")
#seachSWVulnerabilities("19117","1752412")

CVE-2024-21413
CVE-2024-21379
CVE-2024-21378
CVE-2024-20677
CVE-2024-20673
CVE-2023-41764
CVE-2023-36897
CVE-2023-36896
CVE-2023-36895
CVE-2023-36893
CVE-2023-36884
CVE-2023-36866
CVE-2023-36865
CVE-2023-36767
CVE-2023-36766
CVE-2023-36765
CVE-2023-36763
CVE-2023-36762
CVE-2023-36761
CVE-2023-36569
Items already saved
{
    "id": "Office_684235_2024-04-15 20:42:29",
    "partitionKey": "partitionKey",
    "vulnerabilities": {
        "CVE-2024-21413": {
            "summary": "Microsoft Outlook Remote Code Execution Vulnerability",
            "affect": [
                "Microsoft Office 2019",
                "Microsoft 365 Apps for Enterprise",
                "Microsoft Office LTSC 2021",
                "Microsoft Office 2016"
            ],
            "metrics": [
                {
                    "type": "Primary",
                    "source": "secure@microsoft.com",
                    "cvssData": {
                        "scope": "UNCHANGED",
                        "ve

In [18]:
for software in get_installed_software():
    print(software)

{'name': 'Cisco Packet Tracer 8.2.1 64Bit', 'version': '8.2.1.118', 'publisher': 'Cisco Systems, Inc.'}
{'name': 'GDR 1105 para SQL Server 2022 (KB5029379) (64-bit)', 'version': '16.0.1105.1', 'publisher': 'Microsoft Corporation'}
{'name': 'GDR 1110 para SQL Server 2022 (KB5032968) (64-bit)', 'version': '16.0.1110.1', 'publisher': 'Microsoft Corporation'}
{'name': 'GDR 1115 para SQL Server 2022 (KB5035432) (64-bit)', 'version': '16.0.1115.1', 'publisher': 'Microsoft Corporation'}
{'name': 'Microsoft Azure Compute Emulator - v2.9.7', 'version': '2.9.8999.43', 'publisher': 'Microsoft Corporation'}
{'name': 'Microsoft SQL Server 2022 (64-bit)', 'version': '', 'publisher': ''}
{'name': 'Microsoft SQL Server 2022 (64-bit)', 'version': '', 'publisher': 'Microsoft Corporation'}
{'name': '', 'version': '', 'publisher': 'Microsoft Corporation'}
{'name': '', 'version': '', 'publisher': 'Microsoft Corporation'}
{'name': 'Aplicaciones de Microsoft 365 para empresas - es-es', 'version': '16.0.17425